In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [21]:
# Calculate MAPE values
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

In [22]:
# Load the data
data = pd.read_csv('C:\\Users\\tejas\\Desktop\\sample_1 daily.csv', parse_dates=['point_timestamp'], index_col='point_timestamp')

# Set the frequency to daily
data.index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')

data = data.drop('Unnamed: 0', axis = 1)


train_data = data.iloc[:int(len(data)*0.8)] #First 80 % is train data

In [28]:
# Arima 

# last 20% of the data as testing data
test_data = data.iloc[int(len(data) * 0.8):]

# Fit the model
arima_model = sm.tsa.arima.ARIMA(train_data['point_value'], order=(1,1,1)) # using an autoregressive term of order 1, a differencing term of order 1, and a moving average term of order 1
arima_model = arima_model.fit()

# Generate predictions I am using the last 20% of the data as testing data

arima_pred = arima_model.forecast(len(test_data))

# Calculate mean absolute percentage error (MAPE)
arima_mape = calculate_mape(test_data['point_value'], arima_pred)
print('MAPE:', arima_mape)

MAPE: 4.4752859491734


In [30]:
# SARIMA

# last 20% of the data as testing data
test_data = data.iloc[int(len(data) * 0.8):]

# Fit the SARIMA model
sarima_model = sm.tsa.statespace.SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(0, 0, 0, 7)) # 7 indicates that there is a daily seasonality in the data
sarima_model = sarima_model.fit()

# Make predictions
sarima_pred = sarima_model.forecast(len(test_data))

# Calculate MAPE
sarima_mape = calculate_mape(test_data['point_value'], sarima_pred)
print('MAPE:', sarima_mape)

MAPE: 4.4752859491734


In [31]:
# XGBoost

# last 20% of the data as testing data
test_data = data[int(len(data) * 0.8):]

# Define the features and target variable
features = ['point_value']
target = 'point_value'

# Train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(train_data[features], train_data[target])

# Make predictions on the test set
xgb_preds = xgb_model.predict(test_data[features])

# Calculate MAPE
xgb_mape = calculate_mape(test_data[target], xgb_preds)
print('MAPE:', xgb_mape)

MAPE: 0.00035922799334430156


In [33]:
# ETS

# last 20% of the data as testing data
test_data = data[int(len(data) * 0.8):]

# Create and fit the ETS model
ets_model = ETSModel(train_data['point_value'].astype(float), error='add', trend='add', seasonal='add', seasonal_periods=7)
ets_model = ets_model.fit()


# Make predictions on the test set
ets_pred = result.predict(start=test_data.index[0], end=test_data.index[-1])

# Calculate MAPE
ets_mape = calculate_mape(test_data['point_value'], ets_pred)
print('MAPE:', ets_mape)

MAPE: 5.818020742178061


In [34]:
# Selection Algorithm

# Choose the model with the lowest MAPE
mape_values = {"ARIMA": arima_mape, "SARIMA": sarima_mape, "XGBoost": xgb_mape, "ETS": ets_mape}
best_model = min(mape_values, key=mape_values.get)

print("The best model is: ", best_model)


The best model is:  XGBoost
